In [52]:
%matplotlib inline
import pandas as pd
from config import gkey
import json
import csv
import requests
import gmaps
import numpy as np
from scipy.stats import linregress
from matplotlib import pyplot as plt

In [53]:
csv_loc = ('../script1/MapData.csv')
new_csv = pd.read_csv(csv_loc)
new_csv = new_csv.drop('Unnamed: 0', 1)
new_csv

,City Name,Tempurature,Latitude,Longitude,Wind Spd,Humidity %,Cloud %
0,Ushuaia,43.57,-54.80,-68.30,41.38,93,90
1,Narsaq,10.40,60.92,-46.05,9.55,51,62
2,Dikson,-8.95,73.51,80.55,12.17,100,100
3,Longyearbyen,-7.60,78.22,15.64,2.24,83,20
4,Qaanaaq,-13.72,77.48,-69.36,4.76,46,0
...,...,...,...,...,...,...,...
522,Jose Bonifacio,93.20,-21.05,-49.69,6.93,34,20
523,Rio Gallegos,55.40,-51.62,-69.22,39.15,32,75
524,Pasni,52.20,25.26,63.47,27.29,40,0
525,Dakar,71.47,14.69,-17.44,6.93,73,100


In [54]:
gmaps.configure(api_key=gkey)
locations = new_csv[['Latitude', 'Longitude']].astype(float)
humidity = new_csv["Humidity %"].astype(float)
hummax = humidity.max()
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=hummax,
                                 point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [143]:
new_csv = new_csv.loc[new_csv["Tempurature"] >= 70, :]
new_csv = new_csv.loc[new_csv["Tempurature"] <= 80, :]
new_csv = new_csv.loc[new_csv["Wind Spd"] <= 10, :]
new_csv = new_csv.loc[new_csv["Cloud %"] <= 0, :]
new_csv = new_csv.loc[new_csv["Humidity %"] <= 50, :]
new_csv1 = new_csv.reset_index()
new_csv1

,level_0,index,City Name,Tempurature,Latitude,Longitude,Wind Spd,Humidity %,Cloud %
0,0,78,Daoukro,77.50,7.06,-3.96,0.69,28,0
1,1,113,Springbok,79.07,-29.66,17.89,7.56,38,0
2,2,174,Kibala,73.40,9.11,18.35,4.18,49,0
3,3,196,Lola,74.07,7.80,-8.53,2.51,31,0
4,4,219,Bondo,73.11,3.81,23.69,1.32,35,0
5,5,253,Moyale,75.42,3.52,39.06,9.28,44,0
6,6,320,Neuquen,77.00,-38.95,-68.06,9.17,14,0
7,7,331,Rafai,74.77,4.95,23.92,0.81,32,0
8,8,334,Isiro,72.99,2.77,27.62,1.90,41,0
9,9,344,Tambura,73.83,5.60,27.47,1.86,26,0


In [153]:
hotel_name = []

for x in range (0, len(new_csv)):
    lat1 = new_csv1["Latitude"][x]
    lng1 = new_csv1["Longitude"][x]
    target_type = "hotel"
    target_radius = 5000
    location = f"{lat1}, {lng1}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {"location": location, "radius": target_radius, "type": target_type, "key": gkey}

    response = requests.get(base_url, params=params)
    places = response.json()
    hotel_name.append(places["results"][0]["name"])
    


In [154]:
new_csv1["Hotel Name"] = hotel_name

marker_locations = new_csv1[['Latitude', 'Longitude']]
hname = new_csv1["Hotel Name"].tolist()

markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Hotel Name: {x}" for x in hname])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))